In [ ]:
!pip install -q langchain openai faiss-cpu pypdf sentence-transformers PyPDF2 gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00


In [ ]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import gradio as gr

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# List of PDF files
pdf_files = ["part3.pdf"]

# Load text from PDFs
text = load_pdfs(pdf_files)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for the text chunks
embeddings = model.encode(texts)

# Convert embeddings to a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self.get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Load a Hugging Face model with increased max_length
generator = pipeline(
    'text-generation',
    model='gpt2',
    max_length=500,  # Increase max_length
    max_new_tokens=200  # Alternatively, set max_new_tokens
)
llm = HuggingFacePipeline(pipeline=generator)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever
)

# Truncate the input to a maximum number of tokens
def truncate_input(text, max_tokens=500):
    tokens = text.split()  # Simple whitespace-based tokenization
    truncated_tokens = tokens[:max_tokens]
    return " ".join(truncated_tokens)

# Chatbot function with truncation
def chatbot(query):
    truncated_query = truncate_input(query, max_tokens=500)
    response = qa_chain.run(truncated_query)
    return response

# Test the chatbot
query = "What is right to freedom?"
result = chatbot(query)
print("Response:", result)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-5-0f0188630c0d>:69: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=generator)
<ipython-input-5-0f0188630c0d>:87: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain.run(truncated_query)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select 

Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Right to Freedom
19. (1) All citizens shall have the right—
(a) to freedom of speech and expression;
(b) to assemble peaceably and without arms;
(c) to form associations or unions;
(d) to move freely throughout the territory of India;
(e) to reside and settle in any part of the territory
of India;  
1[and]
       2*                       *                      *                      *                      *
(g) to practise any profession, or to carry on any
occupation, trade or business.
3[(2) Nothing in sub-clause ( a) of clause (1) shall affect
the operation of any existing law, or prevent the Statefrom making any law, in so far as such law imposes
reasonable restrictions on the exercise of the right
conferred by the said sub-clause in the interests of 
4[the
sovereignty and integrity of India,] the security of t

In [ ]:
import gradio as gr

# Define the Gradio interface
def gradio_chatbot(query):
    return chatbot(query)

iface = gr.Interface(
    fn=gradio_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your legal query here..."),
    outputs="text",
    title="Legal Advice Chatbot",
    description="Ask any legal question related to Indian law."
)

# Launch the interface
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bec67bc0827c3ea784.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# not working properly
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# List of PDF files
pdf_files = ["part3.pdf"]

# Load text from PDFs
text = load_pdfs(pdf_files)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

# Load a pre-trained sentence transformer model (lightweight)
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # Lightweight and efficient

# Create embeddings for the text chunks
embeddings = model.encode(texts)

# Convert embeddings to a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self.get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Load a lightweight summarization model
summarization_model_name = "sshleifer/distilbart-cnn-12-6"  # Lightweight summarization model
tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=tokenizer)

# Load a lightweight text generation model
generator = pipeline(
    'text-generation',
    model='distilgpt2',  # Lightweight version of GPT-2
    max_length=200,  # Reduce max_length to save memory
    max_new_tokens=100  # Reduce max_new_tokens to save memory
)
llm = HuggingFacePipeline(pipeline=generator)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever
)

# Summarize the input text
def summarize_text(text, max_length=100):  # Reduce max_length for summarization
    summary = summarizer(text, max_length=max_length, min_length=20, do_sample=False)
    return summary[0]['summary_text']

# Chatbot function with summarization
def chatbot(query):
    # Retrieve relevant documents
    relevant_docs = custom_retriever.get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in relevant_docs])

    # Summarize the combined text
    summarized_text = summarize_text(combined_text)

    # Generate a response using the summarized text
    response = qa_chain.run(summarized_text)
    return response

# Test the chatbot
query = "What is right to freedom?"
result = chatbot(query)
print("Response:", result)

# Define the Gradio interface
def gradio_chatbot(query):
    return chatbot(query)

iface = gr.Interface(
    fn=gradio_chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Enter your legal query here..."),
    outputs="text",
    title="Legal Advice Chatbot",
    description="Ask any legal question related to Indian law. Supports multilingual queries."
)

# Launch the interface
iface.launch()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-7-5c7c00eb89aa>:92: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = custom_retriever.get_relevant_documents(query)
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=100) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Response: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

2Ins. by  the Constitution (First Amendment) Act, 1951, s. 5.
3Ins. by the Constitution (Twenty-fifth Amendment) Act, 1971, s. 3 (w.e.f. 20-4-1972).
4Subs. by the Constitution  (Forty-second Amendment) Act, 1976, s. 4, for “the principles
specified in clause ( b) or clause ( c) of article 39” (w.e.f. 3-1-1977). Section 4 has been declared
invalid by the Supreme Court in  Minerva Mills Ltd. and others vs. Union of India and others
(1980) 2 . S.C.C. 591.
5Subs. by the Constitution (Forty-fourth Amendment) Act, 1978,  s. 8,  for “article 14,
article 19 or article 31” (w.e.f. 20-6-1979).
6In Kesavananda Bharati vs. The State of Kerala, (1973 ) Supp. S.C.R. 1, the Supreme Court
held the provisions in italics to be invalid.Saving of lawsgiving effect tocertain directiveprinciples.THE CONSTITUTION OF INDIA
(Part III. —Fun

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# List of PDF files
pdf_files = ["part3.pdf"]

# Load text from PDFs
text = load_pdfs(pdf_files)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Keep the original model for quality

# Create embeddings for the text chunks
embeddings = model.encode(texts)

# Convert embeddings to a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self.get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Load a lightweight summarization model
summarization_model_name = "sshleifer/distilbart-cnn-12-6"  # Lightweight summarization model
tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=tokenizer)

# Load the GPT-2 text generation model
generator = pipeline(
    'text-generation',
    model='gpt2',  # Keep GPT-2 for quality responses
    max_length=500,  # Increase max_length for better responses
    max_new_tokens=200  # Increase max_new_tokens for better responses
)
llm = HuggingFacePipeline(pipeline=generator)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever
)

# Summarize the input text
def summarize_text(text, max_length=150):  # Summarize for better readability
    summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Chatbot function with summarization of generated response
def chatbot(query):
    # Retrieve relevant documents
    relevant_docs = custom_retriever.get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in relevant_docs])

    # Generate a response using the combined text
    response = qa_chain.run(combined_text)

    # Summarize the generated response for better readability
    summarized_response = summarize_text(response)
    return summarized_response

# Test the chatbot
query = "What is right to freedom?"
result = chatbot(query)
print("Response:", result)


<ipython-input-8-946f0a2848e8>:44: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomRetriever(BaseRetriever, BaseModel):
<ipython-input-8-946f0a2848e8>:44: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class CustomRetriever(BaseRetriever, BaseModel):
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
Device set to use cpu
Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples

IndexError: index out of range in self

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# List of PDF files
pdf_files = ["part3.pdf"]

# Load text from PDFs
text = load_pdfs(pdf_files)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Keep the original model for quality

# Create embeddings for the text chunks
embeddings = model.encode(texts)

# Convert embeddings to a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def _get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self._get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Load a lightweight summarization model
summarization_model_name = "sshleifer/distilbart-cnn-12-6"  # Lightweight summarization model
tokenizer = AutoTokenizer.from_pretrained(summarization_model_name)
summarization_model = AutoModelForSeq2SeqLM.from_pretrained(summarization_model_name)
summarizer = pipeline("summarization", model=summarization_model, tokenizer=tokenizer)

# Load the GPT-2 text generation model
generator = pipeline(
    'text-generation',
    model='gpt2',  # Keep GPT-2 for quality responses
    max_length=500,  # Increase max_length for better responses
    max_new_tokens=200,  # Increase max_new_tokens for better responses
    truncation=True  # Explicitly enable truncation
)
llm = HuggingFacePipeline(pipeline=generator)

# Create the RAG chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever
)

# Summarize the input text
def summarize_text(text, max_length=150):  # Summarize for better readability
    # Truncate input text to avoid exceeding the model's token limit
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)
    summary = summarizer(text, max_length=max_length, min_length=30, do_sample=False)
    return summary[0]['summary_text']

# Chatbot function with summarization of generated response
def chatbot(query):
    # Retrieve relevant documents
    relevant_docs = custom_retriever._get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in relevant_docs])

    # Truncate combined text to avoid exceeding the model's token limit
    combined_text = " ".join(combined_text.split()[:500])  # Truncate to 500 words

    # Generate a response using the combined text
    response = qa_chain.run(combined_text)

    # Summarize the generated response for better readability
    summarized_response = summarize_text(response)
    return summarized_response

# Test the chatbot
query = "What is right to freedom?"
result = chatbot(query)
print("Response:", result)


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
Device set to use cpu
Device set to use cpu
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Both `max_new_tokens` (=200) and `max_length`(=500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Token indices sequence length is longer than the specified maximum sequence length for this model (1792 > 1024). Running this sequence through the model will result in indexing errors


IndexError: index out of range in self

In [ ]:
!pip install groclake==0.1.14

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import gradio as gr
import os
from google.colab import userdata
from groclake.modellake import ModelLake

# Set up Modellake
os.environ['GROCLAKE_API_KEY'] = "202cb962ac59075b964b07152d234b70"
os.environ['GROCLAKE_ACCOUNT_ID'] = "2466cc1f4e5446effb7a9250de957141"
modellake = ModelLake()

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# List of PDF files
pdf_files = ["a2023-45.pdf","a2023-46.pdf","a2023-47.pdf","COI.pdf"]

# Load text from PDFs
text = load_pdfs(pdf_files)

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_text(text)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for the text chunks
embeddings = model.encode(texts)

# Convert embeddings to a FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def _get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self._get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Chatbot function with Modellake integration
def chatbot(query):
    # Retrieve relevant documents
    relevant_docs = custom_retriever._get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in relevant_docs])

    # Use Modellake for text generation
    chat_completion_request = {
        "groc_account_id": "c4ca4238a0b923820dcc509a6f75849b",  # Replace with your account ID
        "messages": [
            {"role": "system", "content": "You are a legal assistant. Provide accurate and concise legal advice based on the following context."},
            {"role": "user", "content": f"Context: {combined_text}\n\nQuestion: {query}"}
        ]
    }

    # Generate response using Modellake
    response = modellake.chat_complete(chat_completion_request)
    return response


FileNotFoundError: [Errno 2] No such file or directory: 'a2023-45.pdf'

In [ ]:

# Test the chatbot
query = "What is punishment for theft according to BNS?"
result = chatbot(query)
print("Response:", result)

Response: {'answer': 'The punishment for theft according to the given context from the legal document depends on the specifics of the theft committed:\n\n1. If the theft involves preparation for causing death, hurt, or restraint, the punishment can be rigorous imprisonment for a term which may extend to ten years, and the offender shall also be liable to a fine (Section 307).\n\n2. If the theft is classified as a petty organised crime, the punishment can be imprisonment for a term which shall not be less than one year but which may extend to seven years, and the offender shall also be liable to a fine (Section 112). For a first-time offender, upon return of the stolen property, the punishment may be community service.\n\n3. If the theft is classified as sn'}


In [ ]:
import pickle
# Save the FAISS index and text chunks
faiss.write_index(index, "faiss_index.index")
with open("text_chunks.pkl", "wb") as f:
    pickle.dump(texts, f)

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from transformers import pipeline
import gradio as gr
import os
from google.colab import userdata
from groclake.modellake import ModelLake
import pickle

# Set up Modellake
os.environ['GROCLAKE_API_KEY'] = "202cb962ac59075b964b07152d234b70"
os.environ['GROCLAKE_ACCOUNT_ID'] = "2466cc1f4e5446effb7a9250de957141"
modellake = ModelLake()

# Load the FAISS index from disk
index = faiss.read_index("faiss_index.index")

# Load the text chunks from disk
with open("text_chunks.pkl", "rb") as f:
    texts = pickle.load(f)

# Load the sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def _get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        # Implement asynchronous retrieval if needed
        return self._get_relevant_documents(query)

# Create the custom retriever
custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)

# Chatbot function with Modellake integration
def chatbot(query):
    # Retrieve relevant documents
    relevant_docs = custom_retriever._get_relevant_documents(query)
    combined_text = " ".join([doc.page_content for doc in relevant_docs])

    # Use Modellake for text generation
    chat_completion_request = {
        "groc_account_id": "c4ca4238a0b923820dcc509a6f75849b",  # Replace with your account ID
        "messages": [
            {"role": "system", "content": "You are a legal assistant. Provide accurate and concise legal advice based on the following context."},
            {"role": "user", "content": f"Context: {combined_text}\n\nQuestion: {query}"}
        ]
    }

    # Generate response using Modellake
    response = modellake.chat_complete(chat_completion_request)
    return response


In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from langchain.schema import BaseRetriever, Document
from pydantic import BaseModel
from typing import List
import gradio as gr
import os

# Set up environment (replace with your actual API keys)
os.environ['GROCLAKE_API_KEY'] = "202cb962ac59075b964b07152d234b70"
os.environ['GROCLAKE_ACCOUNT_ID'] = "2466cc1f4e5446effb7a9250de957141"

# Load PDFs
def load_pdfs(pdf_files):
    text = ""
    for pdf_file in pdf_files:
        reader = PdfReader(pdf_file)
        for page in reader.pages:
            text += page.extract_text()
    return text

# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# FAISS index and retriever
index = None
texts = []

class CustomRetriever(BaseRetriever, BaseModel):
    index: any  # FAISS index
    texts: List[str]  # List of text chunks
    model: any  # Sentence Transformer model
    k: int = 3  # Number of documents to retrieve

    def _get_relevant_documents(self, query: str) -> List[Document]:
        query_embedding = self.model.encode([query])
        distances, indices = self.index.search(np.array(query_embedding), self.k)
        return [Document(page_content=self.texts[i]) for i in indices[0]]

# Initialize custom retriever
custom_retriever = None

# Process uploaded PDFs and update the retriever
def process_uploaded_files(files):
    global texts, index, custom_retriever
    text = load_pdfs(files)
    texts = text_splitter.split_text(text)
    embeddings = model.encode(texts)
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings))
    custom_retriever = CustomRetriever(index=index, texts=texts, model=model, k=3)
    return "Documents processed successfully!"

# Chatbot function with chat continuation
def chatbot(query, chat_history=None):
    global custom_retriever
    if chat_history is None:
        chat_history = []

    try:
        # Retrieve relevant documents
        relevant_docs = custom_retriever._get_relevant_documents(query)
        combined_text = " ".join([doc.page_content for doc in relevant_docs])

        # Prepare messages for the model
        messages = [
            {"role": "system", "content": "You are a legal assistant. Provide accurate and concise legal advice based on the following context."},
            *chat_history,
            {"role": "user", "content": f"Context: {combined_text}\n\nQuestion: {query}"}
        ]

        # Simulate a response (replace with your actual model call)
        response = f"Response to: {query}\n\nContext: {combined_text[:200]}..."  # Placeholder

        # Update chat history
        chat_history.append({"role": "user", "content": query})
        chat_history.append({"role": "assistant", "content": response})

        return response, chat_history

    except Exception as e:
        return f"An error occurred: {str(e)}", chat_history

# Gradio interface
def gradio_chatbot(query, chat_history=None):
    return chatbot(query, chat_history)

# Create Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Legal Assistance Chatbot")
    gr.Markdown("Ask legal questions and get accurate advice based on Indian law.")

    # File upload component
    with gr.Row():
        file_upload = gr.File(label="Upload PDFs", file_count="multiple")
        upload_button = gr.Button("Process Documents")

    # Chat interface
    chatbot_interface = gr.ChatInterface(
        fn=gradio_chatbot,
        textbox=gr.Textbox(placeholder="Ask your legal question here...", label="Input"),
        examples=["What is the punishment for theft in India?", "Explain the Right to Education Act."],
        title="Chat with the Legal Assistant"
    )

    # Link file upload to processing function
    upload_button.click(process_uploaded_files, inputs=file_upload, outputs=gr.Textbox(label="Upload Status"))

# Launch the interface
demo.launch()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:502: UserWarning: <built-in function any> is not a Python type (it may be an instance of an object), Pydantic will allow any object with no validation since we cannot even enforce that the input is an instance of the given type. To get rid of this error wrap the type with `pydantic.SkipValidation`.
  warn(
/usr/local/lib/python3.11/dist-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://530b9b9aa57441bc87.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
